In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [4]:
# Набор данных вопросов и ответов
questions = [
    "Как дела?",
    "Что делаешь?",
    "Что нового?",
    # Добавьте больше вопросов
]

answers = [
    "Прекрасно, спасибо!",
    "Я отвечаю на вопросы.",
    "Не так много, а у вас?",
    # Добавьте соответствующие ответы
]

# Создание TF-IDF векторайзера
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(questions)

def get_response(input_question):
    # Предобработка входного вопроса
    input_question = [input_question]

    # Вычисление TF-IDF вектора для входного вопроса
    input_vector = vectorizer.transform(input_question)

    # Вычисление косинусного сходства между входным вопросом и набором вопросов
    similarities = cosine_similarity(input_vector, X)

    # Находим самый похожий вопрос
    closest = np.argmax(similarities)

    # Возвращаем соответствующий ответ
    return answers[closest]

# Пример использования
user_question = "Как дела у тебя?"
response = get_response(user_question)
print(response)


Прекрасно, спасибо!
